In [2]:
import urllib2

In [35]:
class wordDictionary:
    def __init__(self, url):
        #Turn into dictionary of the form (word category, list of words within that category)
        data = urllib2.urlopen(url)
        self.dictionary = {} 
        for line in data:
            words = line.split(":")
            category = words[0]
            wordList = words[1].split(", ")
            wordList[0] = wordList[0].replace('\t','')
            wordList[-1] = wordList[-1].replace('\r','').replace('\n','')
            self.dictionary[words[0]] = wordList
    
    def getLetters(self, category, position, otherLetters = (None, None, None)):
        letters = []
        firstLetter = otherLetters[0]
        secondLetter = otherLetters[1]
        thirdLetter = otherLetters[2]
        for word in self.dictionary[category]:
            if ((word[0] == firstLetter or firstLetter == None) and (word[1] == secondLetter or secondLetter == None) and \
                (word[2] == thirdLetter or thirdLetter == None)):
                newLetter = word[position]
                if not newLetter in letters: 
                    letters.append(newLetter)
        return letters
            
    def getWords(self, category = None, otherLetters = (None, None, None)):
        words = []
        firstLetter = otherLetters[0]
        secondLetter = otherLetters[1]
        thirdLetter = otherLetters[2]
        for word in self.dictionary[category]:
            if ((word[0] == firstLetter or firstLetter == None) and (word[1] == secondLetter or secondLetter == None) and \
                (word[2] == thirdLetter or thirdLetter == None)):
                words.append(word)
        return words
    
def intersection(L1, L2):
    newL = []
    for e in L1:
        if e in L2:
            newL.append(e)
    return newL
            
        
                
dictionary = wordDictionary("http://slazebni.cs.illinois.edu/fall15/assignment2/wordlist.txt")
print dictionary.dictionary

{'emotion': ['AWE', 'CRY', 'JOY', 'MAD', 'SAD', 'WOE'], 'family': ['DAD', 'KIN', 'MOM', 'SON', 'WED'], 'color': ['BAY', 'DUN', 'HUE', 'LAC', 'JET', 'RED', 'TAN'], 'money': ['ATM', 'BUY', 'PAY', 'TAX'], 'number': ['ONE', 'TWO', 'SIX', 'TEN'], 'adjective': ['ANY', 'ARY', 'AZO', 'BON', 'BUM', 'CIS', 'DEF', 'DUN', 'FEW', 'FIT', 'HIP', 'HOT', 'ICY', 'LAW', 'MIM', 'NEE', 'NTH', 'OLD', 'RED', 'SLY', 'WEE'], 'shape': ['ARC', 'DOT', 'ORB', 'RAY'], 'computer': ['APP', 'BIT', 'BUG', 'BUS', 'CAD', 'CAM', 'CPU', 'DNS', 'FAX', 'GIF', 'HIT', 'JOB', 'KEY', 'RAM', 'RAR', 'ROM', 'TAG', 'URL', 'WEB', 'WWW', 'ZIP'], 'clothing': ['ABA', 'ALB', 'BIB', 'BOA', 'BRA', 'CAP', 'FEZ', 'FUR', 'HAT', 'OBI', 'LID', 'PJS', 'TAM', 'TEE', 'TIE', 'TOP', 'WIG'], 'palindrome': ['AHA', 'BIB', 'BUB', 'DAD', 'DID', 'EVE', 'EWE', 'EYE', 'GAG', 'HAH', 'HUH', 'MOM', 'PEP', 'POP', 'SIS', 'TNT', 'TOT', 'TUT', 'WOW'], 'container': ['BAG', 'BIN', 'BOX', 'CAN', 'CUP', 'TIN'], 'pronoun': ['ANY', 'FEW', 'HER', 'HIS', 'HIM', 'ITS', 'ON

In [47]:
class LetterCSP:
    def __init__(self, url, dictionary):
        data = urllib2.urlopen(url)
        i = 0
        self.valTracker = {}
        self.constraints = {}
        for line in data:
            variableList = []
            if i == 0:
                self.size = int(line)
                for j in range(1,self.size+1):
                    self.valTracker[j] = []
            else:
                labels = line.split(":")
                category = labels[0]
                self.constraints[category] = []
                entries = labels[1].split(",")
                entries[-1] = entries[-1].replace("\r\n","")
                position = 0
                for entry in entries:
                    #constrain possible values
                    key = int(entry)
                    if not self.valTracker[key]: 
                        self.valTracker[key] = dictionary.getLetters(category, position)
                    else:
                        self.valTracker[key] = intersection(self.valTracker[key], dictionary.getLetters(category, position))
                    position += 1
                    
                    variableList.append(key)
                    
                    self.constraints[category].append(int(entry))
                    
#                 #list contraints
#                 j = 0 #variable counter
#                 for variable in variableList:
#                     k = 0
#                     for otherVariable in variableList:
#                         if variable != otherVariable:
#                             '''
#                             Say variable B constrains variable A
#                             (B, category of A and B, A's position in word, B's position in word)
#                             '''
#                             self.constraints[variable].append((otherVariable, category, j, k))
#                         k += 1
#                     j += 1
            i += 1
        

CSP = LetterCSP("http://slazebni.cs.illinois.edu/fall15/assignment2/puzzle1.txt", dictionary)
print CSP.valTracker
print CSP.constraints

{1: ['A', 'N', 'O', 'P', 'Q', 'S', 'Y'], 2: ['A', 'B', 'C', 'D', 'F', 'H', 'I', 'L', 'M', 'N', 'O', 'R', 'S', 'W'], 3: ['A', 'E', 'L', 'R', 'T'], 4: ['A', 'M', 'S', 'W'], 5: ['O', 'A'], 6: ['H', 'K', 'A', 'E', 'M', 'O', 'W', 'Y', 'N', 'T', 'F', 'P', 'Z'], 7: ['E', 'D'], 8: ['R', 'A', 'Y', 'I', 'E', 'O'], 9: ['E', 'W']}
{'emotion': [4, 5, 7], 'body': [3, 8, 9], 'adverb': [1, 5, 9], 'adjective': [2, 3, 9], 'verb': [7, 8, 9], 'interjection': [4, 5, 6]}


In [50]:
class CSPSolver:
    def __init__(self, csp, dictionary):
        self.csp = csp
        self.dictionary = dictionary
        
    def backtrackSearch(self):
        assignment = {}
        for i in range(1, self.csp.size+1):
            assignment[i] = None
        valTracker = self.csp.valTracker
        return self.recursiveBacktrack(assignment, valTracker)
    
    def recursiveBacktrack(self, assignment, valTracker):
        if self.isComplete(assignment):
            return (assignment.copy(),)
        
        solutions = ()
        key = self.pickVariable(assignment, valTracker)
        values = self.orderValues(valTracker[key])
        for value in values:
            if self.checkConsistency(key, value, assignment, valTracker):
                assignment[key] = value
                result = self.recursiveBacktrack(assignment, valTracker)
                if result != None:
                    solutions = solutions + result
                assignment[key] = None
        if solutions == ():
            return None
        else:
            return solutions
                    
    def pickVariable(self, assignment, valTracker):
        for key in valTracker:
            if assignment[key] == None:
                return key
    
    def isComplete(self, assignment):
        for key in assignment:
            if assignment[key] == None:
                return False
        return True
        
    def orderValues(self, List):
        return List[:]
    
    def checkConsistency(self, key, value, assignment, valTracker): #check other two letters for constraints
        validvalues = valTracker[key]
        for category in self.csp.constraints:
            values = validvalues
            categoryValues = self.csp.constraints[category]
            if key == categoryValues[0]:
                values = self.dictionary.getLetters(category, 0, (None, assignment[categoryValues[1]], assignment[categoryValues[2]]))
            elif key == categoryValues[1]:
                values = self.dictionary.getLetters(category, 1, (assignment[categoryValues[0]], None, assignment[categoryValues[2]]))
            elif key == categoryValues[2]:
                values = self.dictionary.getLetters(category, 2, (assignment[categoryValues[0]] ,assignment[categoryValues[1]], None))
            
            validvalues = intersection(validvalues, values)

        if value in validvalues:
            return True
        else:
            return False

#valTracker = {1: ["A", "B", "C"], 2 : ["D", "E", "F"]}
#print backtrackSearch(valTracker)
letterSolver = CSPSolver(CSP, dictionary)
solution = letterSolver.backtrackSearch()
print solution

({1: 'N', 2: 'N', 3: 'E', 4: 'M', 5: 'A', 6: 'N', 7: 'D', 8: 'Y', 9: 'E'}, {1: 'N', 2: 'N', 3: 'E', 4: 'S', 5: 'A', 6: 'Y', 7: 'D', 8: 'Y', 9: 'E'}, {1: 'N', 2: 'W', 3: 'E', 4: 'M', 5: 'A', 6: 'N', 7: 'D', 8: 'Y', 9: 'E'}, {1: 'N', 2: 'W', 3: 'E', 4: 'S', 5: 'A', 6: 'Y', 7: 'D', 8: 'Y', 9: 'E'})
